In [1]:
import pandas as pd
import numpy as np 
import os 
from googletrans import Translator



In [2]:
rev = pd.read_csv('../madrid/reviews_detailed/rev_6p.csv')

In [3]:
rev.head(200)

,listing_id,id,date,reviewer_id,comments,idiomas
0,6369,29428,2010-03-14,84790,Simon and Arturo have the ultimate location in...,en
1,6369,31018,2010-03-23,84338,Myself and Kristy originally planned on stayin...,en
2,6369,34694,2010-04-10,98655,We had a great time at Arturo and Simon's ! A ...,en
3,6369,37146,2010-04-21,109871,I very much enjoyed the stay. \r\nIt's a wond...,en
4,6369,38168,2010-04-26,98901,Arturo and Simon are polite and friendly hosts...,en
...,...,...,...,...,...,...
195,26825,31915907,2015-05-11,27329950,"De reden dat ik er was, was bijscholing. Het h...",nl
196,26825,32637136,2015-05-19,9847981,Augustina is a great host and her flat is an a...,en
197,26825,33294739,2015-05-26,32708453,"Pasé una semana en el apartamento de Agustina,...",es
198,26825,36345086,2015-06-27,19619206,Agustina fue muy amable y una persona muy cono...,es


In [4]:
rev.shape

(679118, 6)

In [5]:
rev.idiomas.unique()

array(['en', 'fr', 'es', 'pt', 'de', 'nl'], dtype=object)

In [6]:
rev.comments[195]

'De reden dat ik er was, was bijscholing. Het huis van Augustina is héérlijk, want het is schoon, stil, ligt niet te ver uit het centrum en zij laat je gewoon helemaal met rust! Goed voor de studie. En minder goed, als je komt voor het gemoedelijke contact en veel gesprekken en gezellig tafelen. Zij spreekt beter engels dan ik spaans. Bravo! Bijzondere vrouw, jammer dat mijn spaans nog zo slecht is. Belangrijk om te weten: in de keuken mag niet gekookt worden. Alleen een broodje smeren mag. Eten mag je alleen in de keuken doen en niet in je kamer.'

In [7]:
import re


In [8]:

# Carga el archivo CSV en un DataFrame de pandas
rev = pd.read_csv('../madrid/reviews_detailed/rev_6p.csv')

# Crea una lista de expresiones regulares para eliminar los caracteres no deseados en cada idioma
regex_list = [
    r'[^\x00-\x7F]', # Caracteres no ASCII en inglés
    r'[^\w\s]', # Caracteres no alfanuméricos en español
    r'[^\w\s]', # Caracteres no alfanuméricos en francés
    r'[^\w\s]', # Caracteres no alfanuméricos en italiano
    r'[^\w\s]', # Caracteres no alfanuméricos en portugués
    r'[^\w\s]', # Caracteres no alfanuméricos en alemán
]

# Crea una lista de patrones de sustitución para reemplazar caracteres no deseados en cada idioma
replace_list = [
    '', # Reemplaza caracteres no ASCII en inglés
    '', # Reemplaza caracteres no alfanuméricos en español
    '', # Reemplaza caracteres no alfanuméricos en francés
    '', # Reemplaza caracteres no alfanuméricos en italiano
    '', # Reemplaza caracteres no alfanuméricos en portugués
    '', # Reemplaza caracteres no alfanuméricos en alemán
]

# Limpia los comentarios para cada idioma en la columna "comments" del DataFrame "rev"
for i, row in rev.iterrows():
    comment = row['comments']
    if type(comment) == str:  # Asegura que la columna no esté vacía
        comment = comment.lower()  # Convierte a minúsculas
        comment = re.sub(regex_list[0], replace_list[0], comment)  # Elimina caracteres no ASCII
        rev.at[i, 'comments'] = comment



In [9]:
rev.to_csv('../madrid/reviews_detailed/rev_caract.csv', index=False)


In [10]:
rev = pd.read_csv('../madrid/reviews_detailed/rev_caract.csv')

In [11]:
import spacy
import pandas as pd

nlp_en = spacy.load('en_core_web_sm')
nlp_fr = spacy.load('fr_core_news_sm')
nlp_es = spacy.load('es_core_news_sm')
nlp_pt = spacy.load('pt_core_news_sm')
nlp_de = spacy.load('de_core_news_sm')
nlp_nl = spacy.load('nl_core_news_sm')

def get_keywords(text, nlp):
    doc = nlp(text)
    keywords = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.pos_ in ['NOUN', 'VERB', 'ADJ']]
    return keywords

def extract_keywords(row):
    idioma = row['idiomas']
    text = row['comments']
    
    if idioma == 'en':
        nlp = nlp_en
    elif idioma == 'fr':
        nlp = nlp_fr
    elif idioma == 'es':
        nlp = nlp_es
    elif idioma == 'pt':
        nlp = nlp_pt
    elif idioma == 'de':
        nlp = nlp_de
    elif idioma == 'nl':
        nlp = nlp_nl
    else:
        return ''
    
    keywords = get_keywords(text, nlp)
    return ' '.join(keywords)


# Aplicar la función extract_keywords() a las primeras 100 filas de su dataframe
rev['comments_short'] = rev.apply(extract_keywords, axis=1)

In [12]:
rev.head()

,listing_id,id,date,reviewer_id,comments,idiomas,comments_short
0,6369,29428,2010-03-14,84790,simon and arturo have the ultimate location in...,en,arturo ultimate location step bus line want vi...
1,6369,31018,2010-03-23,84338,myself and kristy originally planned on stayin...,en,plan stay arturo week plan change arturo open ...
2,6369,34694,2010-04-10,98655,we had a great time at arturo and simon's ! a ...,en,great time arturo cosy apartment wonderful ter...
3,6369,37146,2010-04-21,109871,i very much enjoyed the stay. \r\nit's a wond...,en,enjoy stay wonderful room bath great apartment...
4,6369,38168,2010-04-26,98901,arturo and simon are polite and friendly hosts...,en,polite friendly host provide pleasant convenie...


In [19]:
rev.to_csv('../madrid/listings_detailed/comments_short.csv', index=False)